## Avaliação do desempenho dos algoritimos de SMA e RSI no histórico do S&P500

- __Author:__ Gabriel Senra
- __Date:__ 10/02/2022

In [1]:
import pandas as pd
import yfinance as yf
import ta
import numpy as np

### Leitura do banco de dados (Histórico do S&P500 diário)

In [2]:
df = yf.download('^GSPC', start = '1996-01-01')

[*********************100%***********************]  1 of 1 completed


In [34]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA200,RSI,Signal
Date,,,,,,,,,
1996-01-02,615.929993,620.739990,613.169983,620.729980,620.729980,364180000,NaN,NaN,False
1996-01-03,620.729980,623.250000,619.559998,621.320007,621.320007,468950000,NaN,NaN,False
1996-01-04,621.320007,624.489990,613.960022,617.700012,617.700012,512580000,NaN,NaN,False
1996-01-05,617.700012,617.700012,612.020020,616.710022,616.710022,437110000,NaN,NaN,False
1996-01-08,616.710022,618.460022,616.489990,618.460022,618.460022,130360000,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2022-08-08,4155.930176,4186.620117,4128.970215,4140.060059,4140.060059,3604650000,4335.983599,67.258849,False
2022-08-09,4133.109863,4137.299805,4112.089844,4122.470215,4122.470215,3337150000,4333.847051,63.557422,False
2022-08-10,4181.020020,4211.029785,4177.259766,4210.240234,4210.240234,3998590000,4332.173752,72.077083,False


### Calculo dos indicadores e sinal de compra

In [35]:
df['SMA200'] = ta.trend.sma_indicator(df.Close, window=200)

In [36]:
df['RSI'] = ta.momentum.rsi(df.Close, window=10)

In [37]:
df['Signal'] = np.where((df.Close > df.SMA200) & (df.RSI < 30), True, False)

In [38]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA200,RSI,Signal
Date,,,,,,,,,
1996-01-02,615.929993,620.739990,613.169983,620.729980,620.729980,364180000,NaN,NaN,False
1996-01-03,620.729980,623.250000,619.559998,621.320007,621.320007,468950000,NaN,NaN,False
1996-01-04,621.320007,624.489990,613.960022,617.700012,617.700012,512580000,NaN,NaN,False
1996-01-05,617.700012,617.700012,612.020020,616.710022,616.710022,437110000,NaN,NaN,False
1996-01-08,616.710022,618.460022,616.489990,618.460022,618.460022,130360000,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2022-08-08,4155.930176,4186.620117,4128.970215,4140.060059,4140.060059,3604650000,4335.983599,67.258849,False
2022-08-09,4133.109863,4137.299805,4112.089844,4122.470215,4122.470215,3337150000,4333.847051,63.557422,False
2022-08-10,4181.020020,4211.029785,4177.259766,4210.240234,4210.240234,3998590000,4332.173752,72.077083,False


### Identificação das datas de compra e venda

In [39]:
Buying_dates = []
Selling_dates = []

for i in range(len(df)):
    if df.Signal.iloc[i]:
        Buying_dates.append(df.iloc[i+1].name)
        for j in range(1,11):
            if df['RSI'].iloc[i + j] > 40:
                Selling_dates.append(df.iloc[i+j+1].name)
                break
            elif j == 10:
                Selling_dates.append(df.iloc[i+j+1].name)

In [40]:
frame = pd.DataFrame({'Buying_dates':Buying_dates, 'Selling_dates':Selling_dates})
actualtrades = frame[frame.Buying_dates > frame.Selling_dates.shift(1)]
actualtrades = frame[:1].append(actualtrades)

actualtrades.tail()

,Buying_dates,Selling_dates
72,2019-10-03,2019-10-07
73,2020-02-26,2020-03-05
75,2020-10-29,2020-11-04
76,2021-09-21,2021-09-24
78,2022-01-20,2022-02-01


### Rentabilidade em valores absolutos

In [41]:
#A partir das datas de compra e venda vemos a rentabilidade em valores absolutos
profits = df.loc[actualtrades.Selling_dates].Open.values - df.loc[actualtrades.Buying_dates].Open.values
profits

array([ 10.55999756,   9.        ,  17.06994629,  44.85998535,
        11.55004883,  20.35998535,  22.35009766,  34.29992676,
        16.9699707 ,   6.44006348,   9.70007324,   2.45996094,
         3.25      ,   8.68994141,  26.7800293 ,  29.34997559,
        20.04003906,   1.55993652,  28.63000488,  -7.10998535,
        13.10998535,   7.88000488,  32.19006348,   3.63000488,
        32.27001953,  29.60986328,  31.11987305,   9.90991211,
        36.42993164,  20.86010742,  34.79980469,  35.67993164,
        69.33007812,  58.85009766, -64.19995117, 129.29003906,
        63.58984375, -27.78027344])

### Porcentagem de transações Lucrativas:

In [42]:
porcentagemSucesso = round((len([i for i in profits if i > 0])/len (profits))*100,1)
print(porcentagemSucesso,"% das transações foram lucrativas")

92.1 % das transações foram lucrativas


### Porcentagem da Rentabilidade por transação

In [43]:
relprofts = (df.loc[actualtrades.Selling_dates].Open.values-df.loc[actualtrades.Buying_dates].Open.values)/df.loc[actualtrades.Buying_dates].Open.values
relprofts

array([ 0.01464673,  0.01188715,  0.02314098,  0.05115222,  0.01077309,
        0.01585175,  0.01712218,  0.02521757,  0.01533274,  0.00592397,
        0.00827856,  0.0019366 ,  0.00232369,  0.00595638,  0.0249374 ,
        0.02604164,  0.0158845 ,  0.00121272,  0.02106727, -0.00531246,
        0.00796084,  0.00441952,  0.01845951,  0.00188103,  0.01624294,
        0.01396125,  0.01482746,  0.0042484 ,  0.01393231,  0.00788152,
        0.01253202,  0.01265192,  0.02423129,  0.02039596, -0.0204465 ,
        0.03945173,  0.01453665, -0.00610911])

### Rentabilidade média por transação

In [44]:
realProf = round(relprofts.mean()*100,2)
print(realProf,"% de lucro em média por transação")

1.3 % de lucro em média por transação


### Rentabilidade acumulada

In [45]:
rentAcumulada = (relprofts + 1).cumprod()
print(rentAcumulada,"\n")

[1.01464673 1.02670799 1.05046702 1.10420074 1.1160964  1.13378848
 1.15320141 1.18228234 1.20040997 1.20752116 1.2175177  1.21987554
 1.22271015 1.22999307 1.26066591 1.29349572 1.31404225 1.31563581
 1.34335268 1.33621617 1.34685357 1.35280601 1.37777815 1.38036979
 1.40279106 1.42237578 1.443466   1.44959842 1.46979468 1.4813789
 1.49994358 1.51892074 1.55572615 1.58745668 1.55499876 1.61634615
 1.63984241 1.62982443] 



In [46]:
print(round((rentAcumulada[-1]-1)*100,2),"% de lucro acumulado utilizando como indicadores o SMA e o RSI no periodo analisado")

62.98 % de lucro acumulado utilizando como indicadores o SMA e o RSI no periodo analisado


In [57]:
baseLine = round((df['Open'][-1]/df['Open'][0])*100,1)
print("BaseLine:", baseLine,"%")

BaseLine: 686.0 %
